In [ ]:
from NetworkClasses import *
import numpy as np
#Criss-Cross Network
rng = np.random.default_rng(10)
def server1_dist(job):
    if job.cls=="J1":
        mu = 3
        return rng.exponential(1.0 / mu)
    if job.cls=="J3":
        mu = 2
        return rng.exponential(1.0 / mu)
def server2_dist(job):
    mu = 5
    return rng.exponential(1.0 / mu)

S1_servers = [Server(f"S1-s{i}", server1_dist) for i in range(2)]
S1_queues = {"Q1": Queue("S1","Q1"), "Q2": Queue("S1","Q2")}
S1 = Station("S1",S1_queues, S1_servers)
S2_servers = [Server(f"S2-s{i}", server2_dist) for i in range(2)]
S2_queues = {"Q1": Queue("S2","Q1")}
S2 = Station("S2",S2_queues, S2_servers)

arrival_rate = 5
interarrival = exp_interarrival(arrival_rate,rng)
ap = ArrivalProcess("S1", "Q1",interarrival, job_class = "J1")

class CrissCrossRouter():
        def route(self, job: Job, station_id: str, t: float) -> Optional[Tuple[str, str]]:
            if job.cls == "J1":
                job.cls = "J2"
                return ("S2", "Q1")
            if job.cls == "J2":
                job.cls = "J3"
                return ("S1", "Q2")
            else:
                return None
                
router = CrissCrossRouter()

policy_list = [FIFOPolicy(), MaxWeightByQLenPolicy()]
policy = MaxWeightByQLenPolicy()
net = Network(
    stations={"S1": S1, "S2": S2},
    arrivals=[ap],
    router=router,
    policy=policy,
    rng=rng
)

net.run(until_jobs=5000)

print(f"Completed jobs: {net.completed_jobs}")
print(f"Mean sojourn time: {net.mean_sojourn():.4f}")